In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, time
import numpy as np
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications import VGG16

# Load data
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Convert the images into 3 channels
train_images=np.dstack([train_images] * 3)
test_images=np.dstack([test_images] * 3)

train_images = train_images.reshape(-1, 28, 28, 3)
test_images = test_images.reshape(-1, 28, 28, 3)

# Resize the images 50*50 as required by VGG16
train_images = np.asarray([keras.preprocessing.image.img_to_array(keras.preprocessing.image.array_to_img(im, scale=False).resize((50,50))) for im in train_images])
test_images = np.asarray([keras.preprocessing.image.img_to_array(keras.preprocessing.image.array_to_img(im, scale=False).resize((50,50))) for im in test_images])


train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# SCALE THIS data DURING 0 AND 1
train_images = train_images / 255.0
test_images = test_images / 255.0

#train_images.shape, test_images.shape, train_labels.shape, test_labels.shape

def train_seq_model(model, train_images, train_labels, test_images, test_labels, epochs, batch) :
    model.summary()
    start = time.time()
    model.fit(train_images, train_labels, epochs=epochs, batch_size=batch)
    train_time = time.time() - start
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print('Test accuracy:', test_acc)
    print('Train time:', train_time)



In [0]:
#Эксперимент №1
#Использование структуры глубокой модели,
#построенной для решения задачи А,
#с целью обучения аналогичной модели для решения задачи В


IMG_WIDTH = 50
IMG_HEIGHT = 50
IMG_DEPTH = 3
BATCH_SIZE = 128
LEARNING_RATE = 1e-04


f_model = keras.Sequential()
f_model.add(VGG16(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)))
f_model.add(keras.layers.Flatten())
f_model.add(keras.layers.Dense(512, activation='sigmoid'))
f_model.add(keras.layers.Dense(10, activation='softmax'))

sgd = keras.optimizers.SGD(lr=LEARNING_RATE) 
f_model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

train_seq_model(f_model, train_images, train_labels, test_images, test_labels, epochs=20, batch = BATCH_SIZE) 


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_10 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                5130      
Total params: 14,982,474
Trainable params: 14,982,474
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
60000/60000 [==============================] - 32s 538us/step - loss: 2.3635 - acc: 0.1817
Epoch 2/20
60000/60000 [==============================] - 31s 515us/step - loss: 1.9897 - acc: 0.3665
Epoch 3/20
60000/60000 [===========

In [0]:
#Эксперимент №2
#Использование модели, построенной для решения задачи А,
#в качестве фиксированного метода извлечения признаков 
#при построении модели, решающей задачу В


IMG_WIDTH = 50
IMG_HEIGHT = 50
IMG_DEPTH = 3
BATCH_SIZE = 128
LEARNING_RATE = 1e-04


vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH))
#
vgg16.trainable = True
    
s_model = keras.Sequential()
s_model.add(vgg16)
s_model.add(keras.layers.Flatten())
s_model.add(keras.layers.Dense(512, activation='sigmoid'))
s_model.add(keras.layers.Dense(10, activation='softmax'))

adam = keras.optimizers.Adam(lr=LEARNING_RATE) 
s_model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


train_seq_model(s_model, train_images, train_labels, test_images, test_labels, epochs=15, batch = BATCH_SIZE) 

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_11 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_22 (Dense)             (None, 10)                5130      
Total params: 14,982,474
Trainable params: 14,982,474
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
60000/60000 [==============================] - 35s 580us/step - loss: 0.4565 - acc: 0.8357
Epoch 2/15
60000/60000 [==============================] - 33s 547us/step - loss: 0.2921 - acc: 0.8930
Epoch 3/15
60000/60000 [===========

In [0]:
#Эксперимент №3
#Тонкая настройка параметров модели,
#построенной для решения исходной задачи А, с целью решения задачи В


IMG_WIDTH = 50
IMG_HEIGHT = 50
IMG_DEPTH = 3
BATCH_SIZE = 128
LEARNING_RATE = 1e-04


vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH))
  
t_model = keras.Sequential()
t_model.add(vgg16)
t_model.add(keras.layers.Flatten())
t_model.add(keras.layers.Dense(512, activation='sigmoid'))
t_model.add(keras.layers.Dense(10, activation='softmax'))

adam = keras.optimizers.Adam(lr=LEARNING_RATE) 
t_model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


train_seq_model(t_model, train_images, train_labels, test_images, test_labels, epochs=20, batch = BATCH_SIZE) 

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_14 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_28 (Dense)             (None, 10)                5130      
Total params: 14,982,474
Trainable params: 14,982,474
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
60000/60000 [==============================] - 37s 614us/step - loss: 0.4651 - acc: 0.8331
Epoch 2/20
60000/60000 [==============================] - 34s 563us/step - loss: 0.2954 - acc: 0.8921
Epoch 3/20
60000/60000 [===========